In [36]:
import tensorflow.keras as keras
import pickle
from pathlib import Path
from src.utils.preprocessing import preprocess_text


# Data Wrangling & Preprocessing
Using src/utils/make_datasets.py, I have aggregated the text files into a pandas dataframe. X is the review text, and y is the sentiment, where 1 = positive and 0 = negative. The dataset has been split into training, validation and testing data (70%, 10%, 20% respectively). These have been serialized to a pickle file.

Load the data pickle:

In [18]:
with Path('./data/imdb.pickle').resolve().open('rb') as f:
    train, val, test = pickle.load(f)
train.head()

,X,y,path
27238,"I was completely bored with this film, melodra...",0,aclImdb/test/neg/43_1.txt
32694,"In yet another case of misleading marketing, t...",0,aclImdb/test/neg/11058_3.txt
32646,I went to see this movie to kill some time. I ...,0,aclImdb/test/neg/10826_4.txt
45158,I don't know what the rest of you guys watch S...,0,aclImdb/train/neg/7373_4.txt
8430,"I recently was lucky enough to see ""The Kite R...",1,aclImdb/test/pos/4520_10.txt


Let's look at a review:

In [19]:
train.X[1]


"I've read that Paolo Sorrintino's inspiration for The Consequences of Love came from simply observing a businessman alone in a Brazilian hotel, and speculating what he might be doing there. The film unfolds to us in a similar way. We view the existence of Titta who has lived alone in a hotel in Switzerland for 8 years. He is secretive and avoids friendship. His life is dominated by order and regularly. Gradually, the truth of his existence is revealed to us until, finally, his world begins to fall apart.<br /><br />The mystery of Titta is central to The Consequences of Love and it works thanks in large part to the superb performance by Toni Servillo. His character becomes all the more intriguing to us as little details are revealed - that he injects heroin once a week, that he has no imagination. Although some here criticise the dialogue for not being naturalistic, that doesn't matter in my view - this is not that sort of film, and I think the script is great. The camera work is also 

This needs some pre-processing. We will remove code tags, expand contractions and remove punctuation.

In [37]:
for df in [train, val, test]:
    df.X = df.X.apply(preprocess_text)

In [38]:
train.X[0]

'never been kissed is a real feel good film if you have not seen it yet then rent it out i am going to buy it when its released because i loved it drew barrymore is excellent again she plays her part well i felt i could relate to this film because of the school days i had were just as bad i thought the plot was well written there was comedy and romance which are the type of films i love i loved the ending because even though you know what is going to happen it still leaves you sitting on the edge of your seat waiting  anticipating'